# Structure vs Surprise in Lana Del Rey Lyrics

This notebook demonstrates how engineered linguistic features are loaded
and explored for songs from *Born to Die* and *Norman Fucking Rockwell!*.
Full feature engineering, machine learning methods, and interpretations
are documented in the README and implemented in the `lanastance` package.

In [1]:
import pandas as pd

df = pd.read_csv("../data/features/song_features.csv")
df.head()

,lexical_entropy,line_end_similarity,third_person_rate,album,song,second_person_rate,repetition_rate,first_person_rate
0,4.4643,0.8409,0.000,aka_lizzy_grant,aka_lizzy_grant_brite_lites.txt,20.408,0.7045,183.673
1,5.8875,0.4500,0.000,aka_lizzy_grant,aka_lizzy_grant_for_K_part_2.txt,120.000,0.2000,93.333
2,5.6428,0.6333,0.000,aka_lizzy_grant,aka_lizzy_grant_gramma.txt,28.302,0.5167,106.132
3,5.0361,0.5143,9.901,aka_lizzy_grant,aka_lizzy_grant_jump.txt,133.663,0.4571,24.752
4,5.0833,0.6286,4.630,aka_lizzy_grant,aka_lizzy_grant_kill_kill.txt,32.407,0.5714,134.259
